## I have used technical indicators RSI, MACD and performed backtesting on my trading algorithm. 


In [2]:
from pandas_datareader import data
import pandas as pd
import numpy as np


In [3]:
portfolio = 500
days = 750
stock_list = ['RELIANCE.NS']

df = pd.DataFrame()
for ticker in stock_list:
    df[ticker]=data.DataReader(ticker,'yahoo',start='1/1/2010')['Adj Close']
prices  = df.head(days)
prices

nav = pd.DataFrame(index=prices.tail(days-14).index)
nav = nav.assign(leftover=np.zeros(days-14), stock=np.zeros(days-14))
nav.iloc[0,0]=portfolio
nav


,leftover,stock
Date,,
2010-01-22,500.0,0.0
2010-01-25,0.0,0.0
2010-01-27,0.0,0.0
2010-01-28,0.0,0.0
2010-01-29,0.0,0.0
...,...,...
2013-01-07,0.0,0.0
2013-01-08,0.0,0.0
2013-01-09,0.0,0.0


In [4]:
 def rsi(df):
    n=14
    differences  = np.diff(df)
    init=differences[:n]
    upgain = init[init>=0].sum()/n
    downloss = -init[init<0].sum()/n
    rs=upgain/downloss
    rsi = 100. - 100./(1.+rs)
    
    if rsi <30.0:
        return int(1)
    elif rsi >70:
        return int(-1)
    else:
        return 0
    

In [5]:
def macd(df):
    prices=df
    avg12 = prices[:12].mean()
    avg26 = prices[:26].mean()
    mul12=2./(12.+1)
    mul26=2./(26.+1)

    ema12=np.zeros_like(prices)
    ema26=np.zeros_like(prices)
    macd=np.zeros_like(prices)
    ema12[:12]=avg12
    ema26[:26]=avg26
    macd=avg12-avg26

    return macd

In [6]:
def bollingerband(df):
    prices=df
    
    avg20 = prices[:20].mean()
    sma=np.zeros_like(prices)
    sma[:20]=avg20
    stddev=np.zeros_like(prices)
    bolu=np.zeros_like(prices)
    bold=np.zeros_like(prices)
    stddev[:20]=prices[:20].std()
    bolu[:20]= avg20 + 2*stddev[0]
    bold[:20]= avg20 - 2*stddev[0]
    
    return bolu[0],bold[0]

In [7]:
signal = 0
previous_signal = 0
for index, row in nav.iloc[1:].iterrows():
    
    signal = np.sign(signal + rsi(prices['RELIANCE.NS'].loc[:index].tail(14)))
    sigmac = macd(prices['RELIANCE.NS'].loc[:index].tail(14))
    leftover = nav.loc[:index].tail(2).head(1).iloc[0,0]
    bolu , bold = bollingerband(prices['RELIANCE.NS'].loc[:index].tail(20))
    if signal == -1:
        nav.loc[index, 'leftover'] = leftover 
        nav.loc[index , 'stock'] = 0  
        continue
    
    if (signal == 1 and previous_signal == 0) and (sigmac > 0) :
        #BUY
        nav.loc[index, 'leftover'] = leftover - prices.loc[index][0]
        nav.loc[index, 'stock'] = prices.loc[index][0]
    
    if signal == 1 and previous_signal == 1:
        #HOLD
        nav.loc[index, 'leftover'] = leftover
        nav.loc[index , 'stock'] = prices.loc[index][0]
    
    if (signal == 0 and previous_signal == 1) and (sigmac < 0) :
        #SELL
        nav.loc[index, 'leftover'] = leftover + prices.loc[index][0]
        nav.loc[index , 'stock'] = 0
    if signal == 0 and previous_signal == 0:
        #Wait
        nav.loc[index, 'leftover'] = leftover 
        nav.loc[index , 'stock'] = 0    
    
    previous_signal = signal
        

In [8]:
nav

,leftover,stock
Date,,
2010-01-22,500.000000,0.000000
2010-01-25,500.000000,0.000000
2010-01-27,63.082031,436.917969
2010-01-28,63.082031,440.999939
2010-01-29,63.082031,444.847931
...,...,...
2013-01-07,628.296204,0.000000
2013-01-08,628.296204,0.000000
2013-01-09,628.296204,0.000000
